# Standards in EHR research

**Author:** Patrick Wu  
**Date:** 2020-12-03    

CSV files were sourced from [Observational Health Data Sciences (OHDSI)](https://ohdsi.org/).   

## Goal: Map **ICD-10-CM** codes to **SNOMED** codes

### Import software libraries

In [1]:
#Use 'pandas' to manipulate data stored in tables (https://pandas.pydata.org/)
import pandas as pd
##To disable truncation
pd.set_option('display.max_colwidth', None)

### Import sample OHDSI data

In [2]:
#Import sample OHDSI CONCEPT table (https://ohdsi.github.io/CommonDataModel/cdm60.html#concept)
##Define a variable and assign the URL for the csv file of interest
sample_concepts_file = 'https://raw.githubusercontent.com/pwatrick/bmif6300_standards_ehr_research/main/data/sample_concepts.csv' 
##Use pandas to import csv file of interest
sample_concepts = pd.read_csv(sample_concepts_file)

#Import sample OHDSI CONCEPT_RELATIONSHIP table (https://ohdsi.github.io/CommonDataModel/cdm60.html#concept_relationship)
sample_mappings_file = 'https://raw.githubusercontent.com/pwatrick/bmif6300_standards_ehr_research/main/data/sample_icd10cm_snomed_mappings.csv'
sample_mappings = pd.read_csv(sample_mappings_file)

#Import sample patient data
sample_patient_file = 'https://raw.githubusercontent.com/pwatrick/bmif6300_standards_ehr_research/main/data/sample_patient.csv'
sample_patient = pd.read_csv(sample_patient_file)

### Look at structure of imported data




In [3]:
sample_concepts.head(2)

,CONCEPT_ID,CONCEPT_NAME,VOCABULARY_ID,CONCEPT_CODE
0,35207064,Other hyperlipidemia,ICD10CM,E78.4
1,201826,Type 2 diabetes mellitus,SNOMED,44054006


In [4]:
sample_mappings.head(2)

,CONCEPT_ID_1,CONCEPT_ID_2,RELATIONSHIP_ID
0,35207064,432867,Maps to
1,35206881,442793,Maps to


In [5]:
sample_patient

,id,icd10cm_code
0,1,E78.4
1,1,I10
2,1,E11.8


### Create dataframe that maps ICD-10-CM to SNOMED

Transform SNOMED dataframe

In [6]:
snomed_codes = sample_concepts[sample_concepts['VOCABULARY_ID'] == 'SNOMED']
snomed_codes = snomed_codes[['CONCEPT_ID', 'CONCEPT_NAME', 'CONCEPT_CODE']].copy()
snomed_codes.columns = ['snomed_concept_id', 'snomed_name', 'snomed_code']

In [7]:
snomed_codes

,snomed_concept_id,snomed_name,snomed_code
1,201826,Type 2 diabetes mellitus,44054006
2,442793,Diabetic complication,74627003
5,320128,Essential hypertension,59621000
6,432867,Hyperlipidemia,55822004


Transform ICD-10-CM dataframe

In [8]:
icd10cm_codes = sample_concepts[sample_concepts['VOCABULARY_ID'] == "ICD10CM"].copy()
icd10cm_codes = icd10cm_codes[['CONCEPT_ID', 'CONCEPT_NAME', 'CONCEPT_CODE']].copy()
icd10cm_codes.columns = ['icd10cm_concept_id', 'icd10cm_name', 'icd10cm_code']

In [9]:
icd10cm_codes

,icd10cm_concept_id,icd10cm_name,icd10cm_code
0,35207064,Other hyperlipidemia,E78.4
3,35207668,Essential (primary) hypertension,I10
4,35206881,Type 2 diabetes mellitus with unspecified complications,E11.8


Merge dataframes

In [10]:
icd10cm_to_snomed = pd.merge(icd10cm_codes, sample_mappings, left_on = ['icd10cm_concept_id'], right_on = ['CONCEPT_ID_1'], how = 'inner')
icd10cm_to_snomed = pd.merge(icd10cm_to_snomed, snomed_codes, left_on = ['CONCEPT_ID_2'], right_on = ['snomed_concept_id'], how = 'inner')
icd10cm_to_snomed = icd10cm_to_snomed[['icd10cm_code', 'icd10cm_name', 'snomed_code', 'snomed_name']].copy()

In [11]:
icd10cm_to_snomed

,icd10cm_code,icd10cm_name,snomed_code,snomed_name
0,E78.4,Other hyperlipidemia,55822004,Hyperlipidemia
1,I10,Essential (primary) hypertension,59621000,Essential hypertension
2,E11.8,Type 2 diabetes mellitus with unspecified complications,74627003,Diabetic complication
3,E11.8,Type 2 diabetes mellitus with unspecified complications,44054006,Type 2 diabetes mellitus


### Map patient's **ICD-10-CM** codes to **SNOMED** codes

In [12]:
sample_patient_snomed = pd.merge(sample_patient, icd10cm_to_snomed, on = ['icd10cm_code'], how = 'inner')

In [13]:
sample_patient_snomed

,id,icd10cm_code,icd10cm_name,snomed_code,snomed_name
0,1,E78.4,Other hyperlipidemia,55822004,Hyperlipidemia
1,1,I10,Essential (primary) hypertension,59621000,Essential hypertension
2,1,E11.8,Type 2 diabetes mellitus with unspecified complications,74627003,Diabetic complication
3,1,E11.8,Type 2 diabetes mellitus with unspecified complications,44054006,Type 2 diabetes mellitus
